In [30]:
from docx import Document
from geojson import Polygon, MultiPolygon, Feature, FeatureCollection, dump
from pyproj import Proj
import utm

In [36]:
doc = Document('data/coords.docx')

In [37]:
table = doc.tables[0]

data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue

    row_data = dict(zip(keys, text))
    data.append(row_data)

[{'NAME': 'JOHN J MCHAMI',
  'Farm size (ha)': '1',
  'Species': 'Pinus patula',
  'Age of woodlot (years)': '6',
  'Number of trees': '974',
  'COORD1': '36L: 0731818\nUTM: 8948260',
  'COORD2': '36L: 0731871\nUTM: 8948880',
  'COORD3': '36L: 0731964\nUTM: 8949975',
  'COORD4': '36L:  0731925\nUTM: 8949031',
  'COORD5': '-',
  'COORD6': '-',
  'COORD7': '-',
  'COORD8': '-',
  'COORD9': '-'},
 {'NAME': 'BRASIUS MKALAWA',
  'Farm size (ha)': '0.3',
  'Species': 'Pinus patula',
  'Age of woodlot (years)': '6',
  'Number of trees': '432',
  'COORD1': '36L\n0732489\nUTM:\n8949408',
  'COORD2': '36L\n0732524\nUTM\n8949388',
  'COORD3': '36L\n0732448\nUTM\n8949316',
  'COORD4': '36L\n0732427\nUTM\n8949355',
  'COORD5': '-',
  'COORD6': '-',
  'COORD7': '-',
  'COORD8': '-',
  'COORD9': '-'},
 {'NAME': 'ERICK NGOLE',
  'Farm size (ha)': '0.7',
  'Species': 'Pinus patula',
  'Age of woodlot (years)': '6',
  'Number of trees': '497',
  'COORD1': '36L\n0731840\nUTM\n8948783',
  'COORD2': '36L\n

In [23]:
myProj = Proj("+proj=utm +zone=36L, +south +ellps=WGS84")

In [38]:
def parseCoord(s):
    s = s.replace("\n", "")
    s = s.replace(":", "")
    s = s.replace(" ", "")
    s = s.replace(".", "")
    s = s.replace("36L", "")

    east, south = s.split("UTM")
    east = east.lstrip()

    coord = utm.to_latlon(float(east), float(south), 36, "L")
    
    return [coord[1], coord[0]]


In [39]:
features = []

for f in data:
    coords = []
    fc = f.copy()
    for i in range(1, 10):
        index = f'COORD{i}'
        coord_string = fc[index]
        fc.pop(index)
        if len(coord_string) > 4:
            coords.append(parseCoord(coord_string))

    if len(coords) > 0:
        coords.append(coords[0])

    polygon = Polygon([coords])
    features.append(Feature(geometry=polygon, properties=fc))

# add more features...
# features.append(...)

feature_collection = FeatureCollection(features)

with open('output/coords_multi.geojson', 'w') as f:
   dump(feature_collection, f)